In [1]:
import pandas as pd
import datetime

In [8]:
root_dir='/Users/robinruhm/Documents/Research/Fire Hydrology/SlaterFireHydrology';
data_dir=root_dir+'/data';
processed_dir =data_dir+'/Processed';
raw_dir=data_dir+'/Raw';

In [62]:
#precip_data_fn=raw_dir+'/salmonrivercavejunct.csv'
output_file=processed_dir+'/IndianCreekFlowData.txt'

In [63]:
#pd.read_csv(precip_data_fn).tail()

In [70]:
#weather_stations=['happycamp', 'orleans','salmonriver','cavejunction']
#precip_data_fns={'happycamp': '/happycampprecipdata.csv',\
#                 'orleans': '/orleansprecipdata.csv',\
#                 'salmonriver':'/salmonrivercavejunct.csv',\
#                 'cavejunction':'/salmonrivercavejunct.csv'}
#station_name = {'happycamp': 'USC00043761',\
#                 'orleans': 'USC00046508',\
#                 'salmonriver':'US1CASK0009',\
#                 'cavejunction':'US1ORJS0024'}

In [71]:
#precip_df = pd.read_csv(raw_dir+precip_data_fns[weather_stations[0]])[['DATE', 'PRCP', 'TMAX','TMIN','TOBS']].set_index('DATE')
#precip_df=precip_df.rename(columns={'PRCP':'PRCP_'+weather_stations[0], 'TMAX':'TMAX_'+weather_stations[0], \
#                          'TMIN':'TMIN_'+weather_stations[0], 'TOBS':'TOBS_'+weather_stations[0]})

In [72]:
#for station in weather_stations[1:]:
#    precip_station_df = pd.read_csv(precip_data_fn)[['STATION','DATE', 'PRCP']].set_index('DATE')
#    precip_station_df=precip_station_df[precip_station_df['STATION']==station_name[station]]
#    precip_station_df=precip_station_df.drop(columns=['STATION'])
#    precip_station_df=precip_station_df.rename(columns={'PRCP':'PRCP_' + station})
#    precip_df=precip_df.join(precip_station_df, how='outer')


In [73]:
#precip_df.head()

In [74]:
river_names=['IndianCreek']#,'SalmonRiver','SuckerCreek']
discharge_map={'IndianCreek': '296197_00060_00003', 'SalmonRiver':'47824_00060','SuckerCreek':'117619_00060'}
#gage_ht_map={'IndianCreek': '16575_00065','SalmonRiver':'47825_00065','SuckerCreek':'117620_00065'}
header_line_num={'IndianCreek': 28}#, 'SalmonRiver':31,'SuckerCreek':29}

In [75]:

begin_date = '2012-01-01'
end_date='2022-12-07'

#output_df = pd.DataFrame({'DATE':pd.date_range(begin_date, end_date)}).set_index('DATE')
#output_df=output_df.join(precip_df.set_index(pd.DatetimeIndex(precip_df.index)), how='outer')

In [78]:
river = 'IndianCreek'
flow_filename=raw_dir+'/'+ river+'FlowData.txt'
dataframe=pd.read_csv(flow_filename,header=header_line_num[river], skiprows=[header_line_num[river]+1],\
        delimiter='\t', low_memory=False)
dataframe.rename(mapper={discharge_map[river]:'discharge_'+river},inplace=True, axis="columns")
flow_df = dataframe[['datetime','discharge_'+river]]
flow_df['discharge_'+river+'_daily_change']=flow_df['discharge_'+river].diff(periods=1)
split_date=flow_df['datetime'].str.split('-', expand=True)
flow_df['year']=split_date[0]
flow_df['month']=split_date[1]
flow_df['day']=split_date[2]
flow_df.drop(columns=['datetime'], inplace=True)


/var/folders/1k/s33lj1kd7px7nbb93v5jzphc0000gn/T/ipykernel_71670/21605811.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow_df['discharge_'+river+'_daily_change']=flow_df['discharge_'+river].diff(periods=1)
/var/folders/1k/s33lj1kd7px7nbb93v5jzphc0000gn/T/ipykernel_71670/21605811.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow_df['year']=split_date[0]


In [79]:
#output_df=precip_df
#for river in river_names:
#    flow_filename=raw_dir+'/'+ river+'FlowData.txt'
#    dataframe=pd.read_csv(flow_filename,header=header_line_num[river], skiprows=[header_line_num[river]+1],\
#                      delimiter='\t', low_memory=False)
#    dataframe.rename(mapper={discharge_map[river]:'discharge_'+river,gage_ht_map[river]:'gage_ht_'+river},inplace=True, axis="columns")
#    flow_df = dataframe[['datetime','discharge_'+river,'gage_ht_'+river]]
#    flow_df['DATE']=[str[:-6] for str in flow_df['datetime']]
#    flow_df.drop(columns=['datetime'])
#    flow_agg_df = flow_df.groupby(by='DATE').agg('mean')
#    flow_agg_df.columns = flow_agg_df.columns.get_level_values(0)
#    flow_agg_df['discharge_'+river+'_daily_change']=flow_agg_df['discharge_'+river].diff(periods=1) #Diff from previous day
#    output_df = output_df.join(flow_agg_df, how='outer')
    

In [80]:
flow_df.head()

,discharge_IndianCreek,discharge_IndianCreek_daily_change,year,month,day
0,391.0,NaN,2012,01,01
1,296.0,-95.0,2012,01,02
2,244.0,-52.0,2012,01,03
3,210.0,-34.0,2012,01,04
4,188.0,-22.0,2012,01,05


In [81]:
flow_df.to_csv(output_file)